In [1]:
# modules
from pathlib import Path
import pandas as pd
import hysevt.events.detection
import hysevt.events.metrics

In [2]:
help(hysevt.events.detection.local_minimum)
print("_"*100)
help(hysevt.events.metrics.get_gauge_data)
print("_"*100)
help(hysevt.events.metrics.calculate_event_metrics)
print("_"*100)
help(hysevt.events.metrics.calc_hysteresis_index)

Help on function local_minimum in module hysevt.events.detection:

local_minimum(path_gauge_data: pathlib.Path, output_file: pathlib.Path, event_station_id: str, event_version_id='AUT', ma_window='3H', filter_SSC_quantile=0.9, keep_ma_gauge_data=False, path_rscript=PosixPath('/Users/skalevag/opt/anaconda3/envs/test_hysevt/lib/python3.11/site-packages/hysevt/events/identify_events_local_minimum.R')) -> pandas.core.frame.DataFrame
    Identifies hydrological events from streamflow data using the local minima method. 
    Then filters out events where suspended sediment data is missing or incomplete.
    
    Args:
        path_gauge_data (pathlib.Path): absolute path to csv file containing gaunging station data
        event_station_id (str): the guaging station id.
        event_version_id (str): the event detection method used. Defaults to "AUT".
        output_file (pathlib.Path): path to csv file where output is to be saved
    
    Returns:
        pd.DataFrame: list of event start 

Load timeseries data from gauge.

In [3]:
my_gauge_data = Path("/Users/skalevag/data/myGauge_SSC_Q.csv")
gauge_data = hysevt.events.metrics.get_gauge_data(my_gauge_data)
gauge_data.head()

,streamflow,suspended_sediment
time,,
2006-05-01 00:00:00,1.65,246.9718
2006-05-01 00:15:00,1.65,247.4734
2006-05-01 00:30:00,1.65,247.9750
2006-05-01 00:45:00,1.65,248.4766
2006-05-01 01:00:00,1.65,248.9782


Set the output file for the events.

In [4]:
output_file_detected_events = my_gauge_data.parent.joinpath("myEvents.csv")
events = hysevt.events.detection.local_minimum(my_gauge_data,
                                      output_file_detected_events,
                                      event_station_id="STA",
                                      event_version_id="AUT",
                                      path_rscript=Path(".").resolve().parent.joinpath("hysevt","events","identify_events_local_minimum.R"))
events.head()

,event_id,start,end
0,STA-AUT-2006-001,2006-05-18 08:45:00,2006-05-19 09:45:00
1,STA-AUT-2006-002,2006-05-28 10:15:00,2006-06-05 09:30:00
2,STA-AUT-2006-003,2006-06-13 09:00:00,2006-06-14 08:45:00
3,STA-AUT-2006-004,2006-06-14 09:00:00,2006-06-15 09:30:00
4,STA-AUT-2006-005,2006-06-15 09:45:00,2006-06-16 09:00:00


In [7]:
event_series = hysevt.events.metrics.get_event_series(events,gauge_data)

In [9]:
annual_sediment_yield = pd.read_csv(
    my_gauge_data.parent.joinpath(
        f"myGauge_annual_sediment_yield.csv"
    ),
    index_col=0,
)
annual_sediment_yield.head(3)

,sediment_yield
year,
2006,238019.900884
2007,193182.061840
2008,152529.063552


In [10]:
annual_streamflow_volume = pd.read_csv(
    my_gauge_data.parent.joinpath(
        f"myGauge_annual_streamflow_volume.csv"
    ),
    index_col=0,
)
annual_streamflow_volume.head(3)

,streamflow
year,
2006,NaN
2007,131903838.0
2008,149596497.0


In [11]:
event_metrics = hysevt.events.metrics.calculate_event_metrics(
    events,
    gauge_data,
    annual_sediment_yield=annual_sediment_yield,
    annual_streamflow_volume=annual_streamflow_volume,
)

In [12]:
event_metrics.head()

,event_id,start,end,duration,seasonal_timing,year,SSY,pSSY,SSC_max,SSC_mean,...,SSC_mean_log,Qtotal_log,Q_max_log,Q_mean_log,Q_max_previous_ratio,time_since_last_event,last_event_SSY_elapsed_time_logratio,last_event_Qtotal_elapsed_time_logratio,last_event_Q_max_elapsed_time_logratio,last_event_SSC_max_elapsed_time_logratio
0,STA-AUT-2006-001,2006-05-18 08:45:00,2006-05-19 09:45:00,25.00,138,2006,1069.088754,0.004492,3736.9221,1079.428631,...,6.984187,13.688048,2.700690,2.270532,NaN,NaN,NaN,NaN,NaN,NaN
1,STA-AUT-2006-002,2006-05-28 10:15:00,2006-06-05 09:30:00,191.25,148,2006,1806.702578,0.007591,2508.0991,483.575956,...,6.181208,14.851739,2.523326,1.408162,0.177364,216.50,1.596971,8.310457,-2.676901,2.848427
2,STA-AUT-2006-003,2006-06-13 09:00:00,2006-06-14 08:45:00,23.75,164,2006,882.157443,0.003706,1874.1661,931.966941,...,6.837297,13.678917,2.635480,2.312174,-0.112154,191.50,2.244371,9.596851,-2.731562,2.572393
3,STA-AUT-2006-004,2006-06-14 09:00:00,2006-06-15 09:30:00,24.50,165,2006,1041.399567,0.004375,1799.2461,996.552409,...,6.904302,13.813636,2.687847,2.416121,-0.052368,0.25,8.168665,15.065212,4.021774,8.922213
4,STA-AUT-2006-005,2006-06-15 09:45:00,2006-06-16 09:00:00,23.25,166,2006,1144.288279,0.004808,1905.8293,1073.529931,...,6.978707,13.844480,2.758109,2.498790,-0.070262,0.25,8.334615,15.199930,4.074142,8.881417


Below is a workaround to calculate the hysteresis index by calling the R-script from python. If these fail, I recommend opening the R-script in R studio and running it "manually". Then join the hysteresis index with the event metrics table.

In [11]:
# calculate hysteresis index with R script (workaround)
hysteresis_index = hysevt.events.metrics.calc_hysteresis_index(my_gauge_data,
                                                               output_file_detected_events,
                                                               path_rscript=Path(".").resolve().parent.joinpath("hysevt","events","hysteresis_index.R"))
# append to event metrics
event_metrics = event_metrics.join(hysteresis_index.drop(columns=["start","end","event_id"]))

In [12]:
event_metrics.head()

,event_id,start,end,duration,seasonal_timing,year,SSY,pSSY,SSC_max,SSC_mean,...,Q_max_log,Q_mean_log,Q_max_previous_ratio,time_since_last_event,last_event_SSY_elapsed_time_logratio,last_event_Qtotal_elapsed_time_logratio,last_event_Q_max_elapsed_time_logratio,last_event_SSC_max_elapsed_time_logratio,SHI,AHI
0,STA-AUT-2006-001,2006-05-18 08:45:00,2006-05-19 09:45:00,25.00,138,2006,1069.088754,0.004492,3736.9221,1079.428631,...,2.700690,2.270532,NaN,NaN,NaN,NaN,NaN,NaN,0.036840,0.577970
1,STA-AUT-2006-002,2006-05-28 10:15:00,2006-06-05 09:30:00,191.25,148,2006,1806.702578,0.007591,2508.0991,483.575956,...,2.523326,1.408162,0.177364,216.50,1.596971,8.310457,-2.676901,2.848427,-0.107477,0.446438
2,STA-AUT-2006-003,2006-06-13 09:00:00,2006-06-14 08:45:00,23.75,164,2006,882.157443,0.003706,1874.1661,931.966941,...,2.635480,2.312174,-0.112154,191.50,2.244371,9.596851,-2.731562,2.572393,0.031142,0.263456
3,STA-AUT-2006-004,2006-06-14 09:00:00,2006-06-15 09:30:00,24.50,165,2006,1041.399567,0.004375,1799.2461,996.552409,...,2.687847,2.416121,-0.052368,0.25,8.168665,15.065212,4.021774,8.922213,0.061862,0.008235
4,STA-AUT-2006-005,2006-06-15 09:45:00,2006-06-16 09:00:00,23.25,166,2006,1144.288279,0.004808,1905.8293,1073.529931,...,2.758109,2.498790,-0.070262,0.25,8.334615,15.199930,4.074142,8.881417,0.004582,-0.015413
